# **Feature Engineering**

In [2]:
import pandas as pd
import numpy as np

In [3]:
data=pd.read_csv('/content/final.csv')
data.head()

,Unnamed: 0,Dicofre,Parish,MF,M,F,MedianRent_per_m2,Area(Km2),Average_monthly_rental_charges,percentage_daily_car_usage,no_of_children_less_than_6_yrs,avg_commute_time,employed_or_student_res_population,feat_amenity_arena,feat_amenity_arts_centre,feat_amenity_auditorium,feat_amenity_bench,feat_amenity_biergarten,feat_amenity_bus_station,feat_amenity_cafe,feat_amenity_car_wash,feat_amenity_charging_station,feat_amenity_clinic,feat_amenity_community_centre,feat_amenity_concert_hall,feat_amenity_courthouse,feat_amenity_dentist,feat_amenity_embassy,feat_amenity_exhibition_center,feat_amenity_fast_food,feat_amenity_fire_station,feat_amenity_fountain,feat_amenity_fuel,feat_amenity_hospital,feat_amenity_kindergarten,feat_amenity_lavoir,feat_amenity_library,feat_amenity_marketplace,feat_amenity_parking,feat_amenity_pharmacy,...,feat_natural_water,feat_natural_wetland,feat_natural_wood,feat_shop_car,feat_shop_clothes,feat_shop_coffee,feat_shop_copyshop,feat_shop_doityourself,feat_shop_hairdresser,feat_shop_hardware,feat_shop_kiosk,feat_shop_legal_services,feat_shop_mall,feat_shop_newsagent,feat_shop_optics,feat_shop_pet,feat_shop_supermarket,feat_shop_ticket,feat_shop_travel_agency,feat_tourism_artwork,feat_tourism_attraction,feat_tourism_guest_house,feat_tourism_hostel,feat_tourism_hotel,feat_tourism_information,feat_tourism_museum,feat_tourism_picnic_site,feat_tourism_viewpoint,Green_Zone_name,no_of_museums,no_of_parking_lots,no_of_tourist_attractions,gz_daily_avg_demand,Resident,daily_avg_people,no_of_buildings,no_of_families,no_of_people_resident,perc_senior_pop,perc_pop_until_19_yo
0,0,131216,"União das freguesias de Aldoar, Foz do Douro e...",244.0,118.0,126.0,9.03,6.27,387.96,NaN,1537.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,ESPAÇO JOGO E RECREIO HOMEM DO LEME,5,28,30,3.728677,1,17780.803014,4783,15352,38012,0.195123,0.196017
1,1,110658,Belém,173.0,91.0,82.0,11.21,7.12,431.90,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,JARDIM AFONSO DE ALBUQUERQUE,93,129,78,10.259113,1,17497.825376,1775,17185,41163,0.161747,0.204917
2,2,110654,Alvalade,356.0,179.0,177.0,11.10,2.48,381.83,49.87,591.0,19.94,5120.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,JARDIM ALAMEDA CIDADE UNIVERSITÁRIA,8,23,4,47.347816,1,17497.825376,1775,17185,41163,0.161747,0.204917
3,3,110655,Areeiro,211.0,106.0,105.0,10.91,8.02,410.26,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,JARDIM ALAMEDA DOM AFONSO HENRIQUES,105,175,84,41.485945,1,17235.051457,5028,10598,27362,0.161501,0.209341
4,4,110657,Avenidas Novas,271.0,135.0,136.0,11.85,3.69,421.45,NaN,NaN,NaN,NaN,0.0,1.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,1.0,21.0,1.0,...,17.0,0.0,2.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,4.0,0.0,2.0,0.0,0.0,JARDIM AMÁLIA RODRIGUES,8,8,4,5.768512,1,7104.740926,2472,6729,15460,0.240815,0.163131


In [9]:
data.isna().sum().sum()
#In total there are 219 null values

219

In [28]:
#As the data is very less filling null values may lead to inconsistent data 
#So let's remove the columns with null values
data = data[data.columns[~data.isnull().any()]]
data=data.drop(columns='Unnamed: 0')
print("Dataframe Dimensions: {0} ".format(data.shape))
#There are 11 cloumns with null values
print("Null value Count: {0} ".format(data.isna().sum().sum()))               #To check if null values still exists

Dataframe Dimensions: (64, 105) 
Null value Count: 0 


In [42]:
#If more tha 90% of the values are zero they may not show positive impact on predicting other attributes
data=data.drop(columns=data.columns[((data==0).mean()>0.9)],axis=1)
data.shape

(64, 33)

In [41]:
#Let's check what are the columns present in the dataset
print(data.columns.values)

['Dicofre' 'feat_amenity_cafe' 'feat_amenity_fast_food'
 'feat_amenity_fountain' 'feat_amenity_parking'
 'feat_amenity_place_of_worship' 'feat_amenity_police'
 'feat_amenity_restaurant' 'feat_amenity_school' 'feat_amenity_toilets'
 'feat_leisure_outdoor_seating' 'feat_leisure_pitch'
 'feat_leisure_playground' 'feat_leisure_swimming_pool'
 'feat_natural_scrub' 'feat_natural_tree' 'feat_natural_water'
 'feat_tourism_attraction' 'feat_tourism_hotel' 'feat_tourism_museum'
 'Green_Zone_name' 'no_of_museums' 'no_of_parking_lots'
 'no_of_tourist_attractions' 'gz_daily_avg_demand' 'Resident'
 'daily_avg_people' 'no_of_buildings' 'no_of_families'
 'no_of_people_resident' 'perc_senior_pop' 'perc_pop_until_19_yo'
 'gz_daily_avg_demand_res']


In [43]:
correlation_mat = data.corr()
correlation_mat['gz_daily_avg_demand'].sort_values(ascending=False)[:10]

gz_daily_avg_demand           1.000000
gz_daily_avg_demand_res       0.834067
Dicofre                       0.257749
feat_natural_tree             0.220971
no_of_buildings               0.166726
feat_leisure_swimming_pool    0.102324
perc_senior_pop               0.061079
daily_avg_people              0.049368
feat_tourism_hotel            0.044994
feat_amenity_parking          0.023018
Name: gz_daily_avg_demand, dtype: float64

In [44]:
green_areas = pd.read_csv('https://raw.githubusercontent.com/SowmyaMadala/WDL-Stage4/main/Data/Attracting%20Population/WDL_Green_Areas_Dataset_V1.txt' ,sep ="|", encoding ="latin-1").rename(columns = {'Number of Museums in influence area' : 'no_of_museums',
                                                                                                                                  'Number of parking lots in influence area' : 'no_of_parking_lots',
                                                                                                                                   'Number of tourist attractions in influence area': 'no_of_tourist_attractions', 
                                                                                                                                   'Green Zones Daily Average Demand':'gz_daily_avg_demand',
                                                                                                                                    'Daily average of people in influence area': 'daily_avg_people',
                                                                                                                                    'Number of buildings in influence area' : 'no_of_buildings',
                                                                                                                                    'Number of families in influence area': 'no_of_families',
                                                                                                                                    'Number of people resident in influence area' : 'no_of_people_resident'})
resident  = green_areas.loc[green_areas['Resident']==1]
resident = resident.rename(columns= {'daily_avg_people': 'daily_avg_people_resident'})
non_resident  = green_areas.loc[green_areas['Resident']==0]
non_resident = non_resident.rename(columns= {'daily_avg_people': 'daily_avg_people_non_resident'})
combined_data = pd.merge(resident, non_resident[['Green_Zone_name', 'daily_avg_people_non_resident']], how = "inner", on = ['Green_Zone_name']).drop(columns=['Resident'])
total_avg_people=combined_data['daily_avg_people_non_resident']+combined_data['daily_avg_people_resident']
combined_data['total_daily_avg_people']=total_avg_people
gz_daily_demand_resident=(combined_data['daily_avg_people_resident']/combined_data['total_daily_avg_people'])*combined_data['gz_daily_avg_demand']
combined_data['gz_daily_avg_demand_res']=gz_daily_demand_resident

In [45]:
data['gz_daily_avg_demand_res']=combined_data['gz_daily_avg_demand_res']

In [46]:
correlation_mat = data.corr()
correlation_mat['gz_daily_avg_demand_res'].sort_values(ascending=False)[:20]

gz_daily_avg_demand_res       1.000000
gz_daily_avg_demand           0.834067
feat_natural_tree             0.216958
perc_pop_until_19_yo          0.200532
feat_leisure_swimming_pool    0.115213
no_of_museums                 0.111314
no_of_buildings               0.108771
feat_tourism_hotel            0.105575
no_of_parking_lots            0.098837
Dicofre                       0.080145
no_of_tourist_attractions     0.078403
feat_amenity_parking          0.055517
daily_avg_people              0.024185
feat_amenity_school           0.002536
feat_amenity_fountain        -0.009097
feat_amenity_police          -0.021213
feat_leisure_pitch           -0.026204
feat_amenity_cafe            -0.044729
feat_tourism_museum          -0.059903
feat_natural_water           -0.061265
Name: gz_daily_avg_demand_res, dtype: float64

In [47]:
#Let's Consider a dataframe with the values that are having comparitively high correlation with green space demand
data1=data[['Green_Zone_name','Dicofre','gz_daily_avg_demand_res','feat_natural_tree','perc_pop_until_19_yo','feat_leisure_swimming_pool','no_of_museums','no_of_buildings','feat_tourism_hotel','no_of_parking_lots']]
data1.head()

,Green_Zone_name,Dicofre,gz_daily_avg_demand_res,feat_natural_tree,perc_pop_until_19_yo,feat_leisure_swimming_pool,no_of_museums,no_of_buildings,feat_tourism_hotel,no_of_parking_lots
0,ESPAÇO JOGO E RECREIO HOMEM DO LEME,131216,1.042953,0.0,0.196017,0.0,5,4783,0.0,28
1,JARDIM AFONSO DE ALBUQUERQUE,110658,2.973259,0.0,0.204917,0.0,93,1775,0.0,129
2,JARDIM ALAMEDA CIDADE UNIVERSITÁRIA,110654,13.722170,0.0,0.204917,0.0,8,1775,0.0,23
3,JARDIM ALAMEDA DOM AFONSO HENRIQUES,110655,30.761585,0.0,0.209341,0.0,105,5028,0.0,175
4,JARDIM AMÁLIA RODRIGUES,110657,1.026792,0.0,0.163131,1.0,8,2472,4.0,8


In [51]:
socio_data=pd.read_csv('https://raw.githubusercontent.com/SowmyaMadala/WDL-Stage4/main/merged_2020.csv')
data1['Dicofre'] = data1.Dicofre.astype(int)
merged=pd.merge(left=data1,right=socio_data,how='inner',left_on='Dicofre',right_on='ParishCode')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [54]:
correlation_mat = merged.corr()
correlation_mat['gz_daily_avg_demand_res'].sort_values(ascending=False)

gz_daily_avg_demand_res           1.000000
feat_natural_tree                 0.244496
perc_pop_until_19_yo              0.193711
no_of_museums                     0.191531
feat_leisure_swimming_pool        0.155063
no_of_parking_lots                0.141607
feat_tourism_hotel                0.130860
Average_monthly_rental_charges    0.079055
ParishCode                        0.075378
Dicofre                           0.075378
no_of_buildings                   0.015653
MedianRent_per_m2                 0.001857
M                                -0.001206
MF                               -0.004031
F                                -0.006730
Area(Km2)                        -0.124422
Name: gz_daily_avg_demand_res, dtype: float64

In [55]:
data2=pd.read_csv('https://raw.githubusercontent.com/McNickSisto/world_data_league/main/stage_4/data/TimingsandPollution.csv')

In [56]:
merged=pd.merge(left=merged,right=data2,how='inner',on='Green_Zone_name')
merged.head()

,Green_Zone_name,Dicofre,gz_daily_avg_demand_res,feat_natural_tree,perc_pop_until_19_yo,feat_leisure_swimming_pool,no_of_museums,no_of_buildings,feat_tourism_hotel,no_of_parking_lots,ParishCode,Parish,MF,M,F,MedianRent_per_m2,Area(Km2),Average_monthly_rental_charges,avg_opening_hours,avg_spend_time(in min),Rating,Opening(Fulltime/Part time),air_quality_index,no2,o3,so2,pm2_5,pm10,co
0,ESPAÇO JOGO E RECREIO HOMEM DO LEME,131216,1.042953,0.0,0.196017,0.0,5,4783,0.0,28,131216,"União das freguesias de Aldoar, Foz do Douro e...",244,118,126,9.03,6.27,387.96,24.00,75,4.4,1,1,0.10,53.64,0.93,8.40,18.65,236.99
1,JARDIM AFONSO DE ALBUQUERQUE,110658,2.973259,0.0,0.204917,0.0,93,1775,0.0,129,110658,Belém,173,91,82,11.21,7.12,431.90,24.00,45,4.4,1,1,0.08,46.49,0.90,9.81,11.40,230.31
2,JARDIM ALAMEDA CIDADE UNIVERSITÁRIA,110654,13.722170,0.0,0.204917,0.0,8,1775,0.0,23,110654,Alvalade,356,179,177,11.10,2.48,381.83,6.25,60,4.8,0,1,0.08,46.49,0.90,9.81,11.40,230.31
3,JARDIM DO CAMPO GRANDE,110654,201.023191,0.0,0.252080,4.0,107,2701,4.0,191,110654,Alvalade,356,179,177,11.10,2.48,381.83,24.00,60,4.4,1,1,0.08,46.49,0.90,9.81,11.40,230.31
4,PQ. INFANTIL HOSPITAL DE SANTA MARIA,110654,3.615978,4.0,0.169625,0.0,6,1753,0.0,21,110654,Alvalade,356,179,177,11.10,2.48,381.83,24.00,45,3.6,1,1,0.08,46.49,0.90,9.81,11.40,230.31


In [57]:
correlation_mat = merged.corr()
correlation_mat['gz_daily_avg_demand_res'].sort_values(ascending=False)

gz_daily_avg_demand_res           1.000000
avg_spend_time(in min)            0.339612
feat_natural_tree                 0.244496
perc_pop_until_19_yo              0.193711
no_of_museums                     0.191531
feat_leisure_swimming_pool        0.155063
no_of_parking_lots                0.141607
feat_tourism_hotel                0.130860
pm10                              0.082761
Average_monthly_rental_charges    0.079055
Dicofre                           0.075378
ParishCode                        0.075378
co                                0.057543
o3                                0.021425
no_of_buildings                   0.015653
MedianRent_per_m2                 0.001857
M                                -0.001206
MF                               -0.004031
F                                -0.006730
no2                              -0.008360
Opening(Fulltime/Part time)      -0.049945
Rating                           -0.076381
avg_opening_hours                -0.077232
air_quality

In [59]:
final_data=merged[['Dicofre','Green_Zone_name','gz_daily_avg_demand_res','avg_spend_time(in min)','feat_natural_tree','perc_pop_until_19_yo','no_of_museums','feat_leisure_swimming_pool','no_of_parking_lots','feat_tourism_hotel']]
final_data.head()

,Dicofre,Green_Zone_name,gz_daily_avg_demand_res,avg_spend_time(in min),feat_natural_tree,perc_pop_until_19_yo,no_of_museums,feat_leisure_swimming_pool,no_of_parking_lots,feat_tourism_hotel
0,131216,ESPAÇO JOGO E RECREIO HOMEM DO LEME,1.042953,75,0.0,0.196017,5,0.0,28,0.0
1,110658,JARDIM AFONSO DE ALBUQUERQUE,2.973259,45,0.0,0.204917,93,0.0,129,0.0
2,110654,JARDIM ALAMEDA CIDADE UNIVERSITÁRIA,13.722170,60,0.0,0.204917,8,0.0,23,0.0
3,110654,JARDIM DO CAMPO GRANDE,201.023191,60,0.0,0.252080,107,4.0,191,4.0
4,110654,PQ. INFANTIL HOSPITAL DE SANTA MARIA,3.615978,45,4.0,0.169625,6,0.0,21,0.0


**Next Steps**
1. Considering a range for selcting the related attributes
2. Checking how the values are distributed and trying to find some intresting facts through visualizations
3. Fixing a model for predicting


